In [ ]:
import requests

url = "https://api.sportradar.com/tennis/trial/v3/en/competitions.json?api_key=7TOzdGr7SikuNauga42FVUh84HA0ZS1mvb3AaF3j"

headers = {"accept": "application/json"}

response = requests.get(url, headers=headers)

print(response.text)

In [ ]:
import requests
import json

url = "https://api.sportradar.com/tennis/trial/v3/en/competitions.json?api_key=7TOzdGr7SikuNauga42FVUh84HA0ZS1mvb3AaF3j"

headers = {"accept": "application/json"}

response = requests.get(url, headers=headers)

print(response.text)

#convert json file to python dict file
data=json.loads(response.text)

competitions = data['competitions']

competitions_list=[]
for comp in competitions:
    competitions_list.append({
        "id" : comp["id"],
        "name" : comp["name"],
        "parent_id": comp.get("parent_id"),
        "type" : comp["type"],
        "gender" : comp["gender"],
        "category_id" : comp["category"]["id"]
    })

category_list=[]
for cate in competitions:
    category_list.append({
        "category_id" : cate["category"]["id"],
        "category_name" : cate["category"]["name"]
    })

In [ ]:
import pandas as pd
competition_df=pd.DataFrame(competitions_list)
competition_df

In [ ]:
category_df=pd.DataFrame(category_list)
category_df

In [ ]:
category_d= category_df.drop_duplicates()
print(category_d)

In [ ]:
category_d.to_csv("category.csv")

In [ ]:
competition_d= competition_df.drop_duplicates()
print(competition_d)

In [ ]:
competition_d.to_csv("competition.csv")

In [ ]:
import mysql.connector

mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    password="",
    database="tennis_data"   
)

mycursor = mydb.cursor(buffered=True)

In [ ]:
mycursor.execute("""
CREATE TABLE Categories ( 
    category_id VARCHAR(50) PRIMARY KEY,
    category_name VARCHAR(100) NOT NULL
)
""")

In [ ]:
mycursor.execute("""
CREATE TABLE Competitions ( 
    competition_id VARCHAR(50) PRIMARY KEY,
    competition_name VARCHAR(100) NOT NULL,
    parent_id VARCHAR(50),
    type VARCHAR(20) NOT NULL,
    gender VARCHAR(10) NOT NULL,
    category_id VARCHAR(50), FOREIGN KEY (category_id) 
    REFERENCES Categories(category_id)                                              
)
""")

In [ ]:
from tabulate import tabulate

In [ ]:
categories_query="INSERT INTO Categories(category_id,category_name) VALUES (%s, %s)"

category_data = [(row['category_id'], row['category_name']) for index, row in category_d.iterrows()]

mycursor.executemany(categories_query, category_data)
mydb.commit()


In [ ]:
mycursor.execute("SELECT * FROM Categories")
out=mycursor.fetchall()
print(tabulate(out, headers=[i[0] for i in mycursor.description],  tablefmt='psql'))

In [ ]:
mycursor.execute("SELECT count(*) FROM Categories")
out=mycursor.fetchall()
print(tabulate(out, headers=[i[0] for i in mycursor.description],  tablefmt='psql'))

In [ ]:
competitions_query="INSERT INTO Competitions(competition_id,competition_name,parent_id,type,gender,category_id) VALUES (%s, %s, %s, %s, %s, %s)"

competition_data=competition_d[['id','name','parent_id','type','gender','category_id']].values.tolist()

mycursor.executemany(competitions_query, competition_data)
mydb.commit()

In [ ]:
mycursor.execute("SELECT * FROM Competitions")
out=mycursor.fetchall()
print(tabulate(out, headers=[i[0] for i in mycursor.description],  tablefmt='psql'))

In [ ]:
mycursor.execute("SELECT count(*) FROM Competitions")
out=mycursor.fetchall()
print(tabulate(out, headers=[i[0] for i in mycursor.description],  tablefmt='psql'))

In [ ]:
# List all competitions along with their category name

mycursor.execute("""
SELECT Competitions.competition_id,Competitions.competition_name,Categories.category_name 
FROM Competitions 
JOIN Categories ON Categories.category_id = Competitions.category_id""")
out=mycursor.fetchall()
print(tabulate(out, headers=[i[0] for i in mycursor.description],  tablefmt='psql'))


In [ ]:
# Count the number of competitions in each category

mycursor.execute("""
SELECT Categories.category_name, COUNT(*) AS competition_count
FROM Competitions
JOIN Categories ON Competitions.category_id = Categories.category_id
GROUP BY Categories.category_name
""")
out=mycursor.fetchall()
print(tabulate(out, headers=[i[0] for i in mycursor.description],  tablefmt='psql'))


In [ ]:
# Find all competitions of type 'doubles'

mycursor.execute("SELECT competition_id,competition_name,parent_id,type,gender FROM Competitions WHERE type='doubles'")
out=mycursor.fetchall()
print(tabulate(out, headers=[i[0] for i in mycursor.description],  tablefmt='psql'))


In [ ]:
# Get competitions that belong to a specific category (e.g., ITF Men)

mycursor.execute("""
SELECT Competitions.competition_id,Competitions.competition_name,Categories.category_name
FROM Competitions
JOIN Categories ON Competitions.category_id = Categories.category_id
WHERE Categories.category_name = "ATP"
""")
out=mycursor.fetchall()
print(tabulate(out, headers=[i[0] for i in mycursor.description],  tablefmt='psql'))

In [ ]:
# Identify parent competitions and their sub-competitions

mycursor.execute("""
SELECT 
parent.competition_name AS parent_competition,              
child.competition_name AS sub_competition
FROM Competitions AS child
JOIN Competitions AS parent ON child.parent_id = parent.competition_id 
""")
out=mycursor.fetchall()
print(tabulate(out, headers=[i[0] for i in mycursor.description],  tablefmt='psql'))

In [ ]:
# Analyze the distribution of competition types by category

mycursor.execute("""
SELECT Competitions.type,Categories.category_name,count(*) AS count_of_comptitions
FROM Competitions 
JOIN Categories ON Competitions.category_id = Categories.category_id
GROUP BY Categories.category_name, Competitions.type
ORDER BY Categories.category_name, Competitions.type;
""")
out=mycursor.fetchall()
print(tabulate(out, headers=[i[0] for i in mycursor.description],  tablefmt='psql'))

In [ ]:
# List all competitions with no parent (top-level competitions)

mycursor.execute("SELECT competition_id,competition_name FROM Competitions WHERE parent_iD IS NULL ")
out=mycursor.fetchall()
print(tabulate(out, headers=[i[0] for i in mycursor.description],  tablefmt='psql'))